In [1]:
import tensorflow as tf
import numpy as np
import scipy
from scipy import misc
import glob
from PIL import Image
import os
import matplotlib.pyplot as plt
import librosa
import librosa.display
from keras import layers
from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, 
                          Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D)
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
import pydot
from IPython.display import SVG, Audio
from keras.layers import Dropout
from multiprocessing import Process
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.initializers import glorot_uniform
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from pydub import AudioSegment
import shutil
import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator
import random
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

2022-11-19 07:42:25.650222: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-19 07:42:27.340407: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-19 07:42:27.356258: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-19 07:42:27.356365: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

In [ ]:
##### THIS IS A BACKUP FILE FOR ALL DEVELOPED DATA RETRIVAL METHODS

In [ ]:
rootdir = "../content"

In [ ]:
# Split 30 sec audio to 3 sec

genres = 'blues classical country disco hiphop jazz metal pop reggae rock'
genres = genres.split()

if not os.path.exists('{rootdir}'):
  os.makedirs('{rootdir}/')
  
if not os.path.exists('{rootdir}/audio3sec'):
  os.makedirs('{rootdir}/audio3sec')

if len(os.listdir(os.path.join('{rootdir}/audio3sec'))) == 0:
  for g in genres:
    path_audio = os.path.join('{rootdir}/audio3sec',f'{g}')
    os.makedirs(path_audio)
  
from pydub import AudioSegment
i = 0
for g in genres:
  #if len(os.listdir(os.path.join('{rootdir}/audio3sec/',f"{g}"))) == 0:
    j=0
    print(f"{g}")
    for filename in os.listdir(os.path.join('{rootdir}/genres',f"{g}")):
      song  =  os.path.join(f'{rootdir}/genres/{g}',f'{filename}')
      j = j+1
      for w in range(0,10):
        i = i+1
        #print(i)
        t1 = 3*(w)*1000
        t2 = 3*(w+1)*1000
        newAudio = AudioSegment.from_wav(song)
        new = newAudio[t1:t2]
        new.export(f'{rootdir}/audio3sec/{g}/{g+str(j)+str(w)}.wav', format="wav")

In [ ]:
# Example of MFCC & MelSpectrogram

y,sr = librosa.load('{rootdir}/audio3sec/blues/blues10.wav',duration=3)

# MFCC
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc= 13, hop_length=512, n_fft=2048)
fig = plt.Figure(figsize=(13, 130))
canvas = FigureCanvas(fig)
p = librosa.display.specshow(mfccs)

# MelSpectrogram
mels = librosa.feature.melspectrogram(y=y,sr=sr)
fig = plt.Figure(figsize=(13, 130))
canvas = FigureCanvas(fig)
p = librosa.display.specshow(librosa.power_to_db(mels, ref=np.max))

In [3]:
# MFCCs for 3 Sec Clips

genres = 'blues classical country disco hiphop jazz metal pop reggae rock'
genres = genres.split()

if not os.path.exists('{rootdir}'):
  os.makedirs('{rootdir}/')
  
if not os.path.exists('{rootdir}/mfccs3sec'):
  os.makedirs('{rootdir}/mfccs3sec')
  os.makedirs('{rootdir}/mfccs3sec/train')
  os.makedirs('{rootdir}/mfccs3sec/test')

  for g in genres:
    path_train = os.path.join('{rootdir}/mfccs3sec/train',f'{g}')
    path_test = os.path.join('{rootdir}/mfccs3sec/test',f'{g}')
    os.makedirs(path_train)
    os.makedirs(path_test)

# Single-Core Implmentation [SLOW]
'''
for g in genres:
  if len(os.listdir(os.path.join('{rootdir}/mfccs3sec/train/',f"{g}"))) == 0:
    j = 0
    print(g)
    for filename in os.listdir(os.path.join('{rootdir}/audio3sec',f"{g}")):
      song  =  os.path.join(f'{rootdir}/audio3sec/{g}',f'{filename}')
      j = j+1
      y,sr = librosa.load(song,duration=3)
      #############################################################################
      mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc= 13, hop_length=512, n_fft=2048)
      fig = plt.Figure(figsize=(13, 130))
      canvas = FigureCanvas(fig)
      p = librosa.display.specshow(mfccs)
      #########################################################
      plt.savefig(f'{rootdir}/mfccs3sec/train/{g}/{g+str(j)}.png')
'''

# Multi-Core Implmentation (Equal For Loop distribution) [Faster]
'''
def task(g):
    j = 0
    print(g)
    for filename in os.listdir(os.path.join('{rootdir}/audio3sec',f"{g}")):
        song  =  os.path.join(f'{rootdir}/audio3sec/{g}',f'{filename}')
        j = j+1
        y,sr = librosa.load(song,duration=3)
        #############################################################################
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc= 13, hop_length=512, n_fft=2048)
        fig = plt.Figure(figsize=(13, 130))
        canvas = FigureCanvas(fig)
        p = librosa.display.specshow(mfccs)
        #########################################################
        plt.savefig(f'{rootdir}/mfccs3sec/train/{g}/{g+str(j)}.png')

processes = [Process(target=task, args=(g,)) for g in genres]

for process in processes:
    process.start()
for process in processes:
    process.join()
'''

# Multi-Core Implmentation (Batches) [Fastest]
def task(g,j,filename):
    song  =  os.path.join(f'{rootdir}/audio3sec/{g}',f'{filename}')
    y,sr = librosa.load(song,duration=3)
    #############################################################################
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc= 13, hop_length=512, n_fft=2048)
    fig = plt.Figure(figsize=(13, 130))
    canvas = FigureCanvas(fig)
    p = librosa.display.specshow(mfccs)
    #########################################################
    plt.savefig(f'{rootdir}/mfccs3sec/train/{g}/{g+str(j)}.png')

batch_size = 32
counter = 0
processes = []

for g in genres:
    j = 0
    print(g)
    for filename in os.listdir(os.path.join('content/audio3sec',f"{g}")):
        if counter <= batch_size:
            j = j+1
            processes = np.append(processes, Process(target=task, args=(g,j,filename)))
            counter += 1
        else:
            for process in processes:
                process.start()
            for process in processes:
                process.join()
            counter = 0
            processes = []
            j = j+1
            processes = np.append(processes, Process(target=task, args=(g,j,filename)))
            counter += 1
            
for process in processes:
    process.start()
for process in processes:
    process.join()

jazz
metal
pop
reggae
rock


In [ ]:
# MFCCs for 30 Sec Clips

genres = 'blues classical country disco hiphop jazz metal pop reggae rock'
genres = genres.split()

if not os.path.exists('{rootdir}'):
  os.makedirs('{rootdir}/')
  
if not os.path.exists('{rootdir}/mfccs30sec'):
  os.makedirs('{rootdir}/mfccs30sec')
  os.makedirs('{rootdir}/mfccs30sec/train')
  os.makedirs('{rootdir}/mfccs30sec/test')

  for g in genres:
    path_train = os.path.join('{rootdir}/mfccs30sec/train',f'{g}')
    path_test = os.path.join('{rootdir}/mfccs30sec/test',f'{g}')
    os.makedirs(path_train)
    os.makedirs(path_test)

# Single-Core Implmentation [SLOW]
'''
for g in genres:
  if len(os.listdir(os.path.join('{rootdir}/mfccs30sec/train/',f"{g}"))) == 0:
    j = 0
    print(g)
    for filename in os.listdir(os.path.join('{rootdir}/genres',f"{g}")):
      song  =  os.path.join(f'{rootdir}/genres/{g}',f'{filename}')
      j = j+1
      y,sr = librosa.load(song,duration=3)
      #############################################################################
      mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc= 13, hop_length=512, n_fft=2048)
      fig = plt.Figure(figsize=(13, 130))
      canvas = FigureCanvas(fig)
      p = librosa.display.specshow(mfccs)
      #########################################################
      plt.savefig(f'content/mfccs30sec/train/{g}/{g+str(j)}.png')
'''

# Multi-Core Implmentation (Equal For Loop distribution) [Faster]
'''
def task(g):
    j = 0
    print(g)
    for filename in os.listdir(os.path.join('content/genres',f"{g}")):
        song  =  os.path.join(f'content/genres/{g}',f'{filename}')
        j = j+1
        y,sr = librosa.load(song,duration=3)
        #############################################################################
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc= 13, hop_length=512, n_fft=2048)
        fig = plt.Figure(figsize=(13, 130))
        canvas = FigureCanvas(fig)
        p = librosa.display.specshow(mfccs)
        #########################################################
        plt.savefig(f'content/mfccs30sec/train/{g}/{g+str(j)}.png')

processes = [Process(target=task, args=(g,)) for g in genres]

for process in processes:
    process.start()
for process in processes:
    process.join()
'''

# Multi-Core Implmentation (Batches) Fastest
def task(g,j,filename):
    song  =  os.path.join(f'content/genres/{g}',f'{filename}')
    y,sr = librosa.load(song,duration=3)
    #############################################################################
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc= 13, hop_length=512, n_fft=2048)
    fig = plt.Figure(figsize=(13, 130))
    canvas = FigureCanvas(fig)
    p = librosa.display.specshow(mfccs)
    #########################################################
    plt.savefig(f'content/mfccs30sec/train/{g}/{g+str(j)}.png')

batch_size = 32
counter = 0
processes = []

for g in genres:
    j = 0
    print(g)
    for filename in os.listdir(os.path.join('content/genres',f"{g}")):
        if counter <= batch_size:
            j = j+1
            processes = np.append(processes, Process(target=task, args=(g,j,filename)))
            counter += 1
        else:
            for process in processes:
                process.start()
            for process in processes:
                process.join()
            counter = 0
            processes = []
            j = j+1
            processes = np.append(processes, Process(target=task, args=(g,j,filename)))
            counter += 1
            
for process in processes:
    process.start()
for process in processes:
    process.join()

In [2]:
# Mel Spectrogram for 3 Sec Clips

genres = 'blues classical country disco hiphop jazz metal pop reggae rock'
genres = genres.split()

if not os.path.exists('{rootdir}'):
  os.makedirs('{rootdir}/')
  
if not os.path.exists('{rootdir}/spectrograms3sec'):
  os.makedirs('{rootdir}/spectrograms3sec')
  os.makedirs('{rootdir}/spectrograms3sec/train')
  os.makedirs('{rootdir}/spectrograms3sec/test')

  for g in genres:
    path_train = os.path.join('{rootdir}/spectrograms3sec/train',f'{g}')
    path_test = os.path.join('{rootdir}/spectrograms3sec/test',f'{g}')
    os.makedirs(path_train)
    os.makedirs(path_test)

# Single-Core Implmentation (Slow)
'''
for g in genres:
  if len(os.listdir(os.path.join('{rootdir}/spectrograms3sec/train/',f"{g}"))) == 0:
    j = 0
    print(g)
    for filename in os.listdir(os.path.join('{rootdir}/audio3sec',f"{g}")):
      song  =  os.path.join(f'{rootdir}/audio3sec/{g}',f'{filename}')
      j = j+1
      y,sr = librosa.load(song,duration=3)
      #############################################################################
      mels = librosa.feature.melspectrogram(y=y,sr=sr)
      fig = plt.Figure(figsize=(13, 130))
      canvas = FigureCanvas(fig)
      p = librosa.display.specshow(librosa.power_to_db(mels, ref=np.max))
      #########################################################
      plt.savefig(f'{rootdir}/spectrograms3sec/train/{g}/{g+str(j)}.png')
'''

# Multi-Core Implmentation (Equal For Loop distribution) [Faster]
'''
def task(g):
    j = 0
    print(g)
    for filename in os.listdir(os.path.join('{rootdir}/audio3sec',f"{g}")):
        song  =  os.path.join(f'{rootdir}/audio3sec/{g}',f'{filename}')
        j = j+1
        y,sr = librosa.load(song,duration=3)
        #############################################################################
        mels = librosa.feature.melspectrogram(y=y,sr=sr)
        fig = plt.Figure(figsize=(13, 130))
        canvas = FigureCanvas(fig)
        p = librosa.display.specshow(librosa.power_to_db(mels, ref=np.max))
        #########################################################
        plt.savefig(f'{rootdir}/spectrograms3sec/train/{g}/{g+str(j)}.png')

processes = [Process(target=task, args=(g,)) for g in genres]

for process in processes:
    process.start()
for process in processes:
    process.join()
'''

# Parallel (Batches) [Fastest]
def task(g,j,filename):
    song  =  os.path.join(f'{rootdir}/audio3sec/{g}',f'{filename}')
    y,sr = librosa.load(song,duration=3)
    #############################################################################
    mels = librosa.feature.melspectrogram(y=y,sr=sr)
    fig = plt.Figure(figsize=(13, 130))
    canvas = FigureCanvas(fig)
    p = librosa.display.specshow(librosa.power_to_db(mels, ref=np.max))
    #########################################################
    plt.savefig(f'{rootdir}/spectrograms3sec/train/{g}/{g+str(j)}.png')

batch_size = 32
counter = 0
processes = []

for g in genres:
    j = 0
    print(g)
    for filename in os.listdir(os.path.join('{rootdir}/audio3sec',f"{g}")):
        if counter <= batch_size:
            j = j+1
            processes = np.append(processes, Process(target=task, args=(g,j,filename)))
            counter += 1
        else:
            for process in processes:
                process.start()
            for process in processes:
                process.join()
            counter = 0
            processes = []
            j = j+1
            processes = np.append(processes, Process(target=task, args=(g,j,filename)))
            counter += 1
            
for process in processes:
    process.start()
for process in processes:
    process.join()

blues
classical
country
disco
hiphop
jazz
metal
pop
reggae
rock


In [ ]:
# Mel Spectrogram for 30 Sec Clips

genres = 'blues classical country disco hiphop jazz metal pop reggae rock'
genres = genres.split()

if not os.path.exists('{rootdir}'):
  os.makedirs('{rootdir}/')
  
if not os.path.exists('{rootdir}/spectrograms30sec'):
  os.makedirs('{rootdir}/spectrograms30sec')
  os.makedirs('{rootdir}/spectrograms30sec/train')
  os.makedirs('{rootdir}/spectrograms30sec/test')

  for g in genres:
    path_train = os.path.join('{rootdir}/spectrograms30sec/train',f'{g}')
    path_test = os.path.join('{rootdir}/spectrograms30sec/test',f'{g}')
    os.makedirs(path_train)
    os.makedirs(path_test)

# Single-Core Implementation
'''
for g in genres:
  if len(os.listdir(os.path.join('{rootdir}/spectrograms30sec/train/',f"{g}"))) == 0:
    j = 0
    print(g)
    for filename in os.listdir(os.path.join('{rootdir}/genres',f"{g}")):
      song  =  os.path.join(f'{rootdir}/genres/{g}',f'{filename}')
      j = j+1
      y,sr = librosa.load(song,duration=3)
      #############################################################################
      mels = librosa.feature.melspectrogram(y=y,sr=sr)
      fig = plt.Figure(figsize=(13, 130))
      canvas = FigureCanvas(fig)
      p = librosa.display.specshow(librosa.power_to_db(mels, ref=np.max))
      #########################################################
      plt.savefig(f'{rootdir}/spectrograms30sec/train/{g}/{g+str(j)}.png')
'''

# Multi-Core Implmentation (Equal For Loop distribution) [Faster]
'''
def task(g):
    j = 0
    print(g)
    for filename in os.listdir(os.path.join('{rootdir}/genres',f"{g}")):
        song  =  os.path.join(f'{rootdir}/genres/{g}',f'{filename}')
        j = j+1
        y,sr = librosa.load(song,duration=3)
        #############################################################################
        mels = librosa.feature.melspectrogram(y=y,sr=sr)
        fig = plt.Figure(figsize=(13, 130))
        canvas = FigureCanvas(fig)
        p = librosa.display.specshow(librosa.power_to_db(mels, ref=np.max))
        #########################################################
        plt.savefig(f'{rootdir}/spectrograms30sec/train/{g}/{g+str(j)}.png')

processes = [Process(target=task, args=(g,)) for g in genres]

for process in processes:
    process.start()
for process in processes:
    process.join()
'''

# Multi-Core Implmentation (Batches) [Fastest]
def task(g,j,filename):
    song  =  os.path.join(f'{rootdir}/genres/{g}',f'{filename}')
    y,sr = librosa.load(song,duration=3)
    #############################################################################
    mels = librosa.feature.melspectrogram(y=y,sr=sr)
    fig = plt.Figure(figsize=(13, 130))
    canvas = FigureCanvas(fig)
    p = librosa.display.specshow(librosa.power_to_db(mels, ref=np.max))
    #########################################################
    plt.savefig(f'{rootdir}/spectrograms30sec/train/{g}/{g+str(j)}.png')

batch_size = 32
counter = 0
processes = []

for g in genres:
    j = 0
    print(g)
    for filename in os.listdir(os.path.join('{rootdir}/genres',f"{g}")):
        if counter <= batch_size:
            j = j+1
            processes = np.append(processes, Process(target=task, args=(g,j,filename)))
            counter += 1
        else:
            for process in processes:
                process.start()
            for process in processes:
                process.join()
            counter = 0
            processes = []
            j = j+1
            processes = np.append(processes, Process(target=task, args=(g,j,filename)))
            counter += 1
            
for process in processes:
    process.start()
for process in processes:
    process.join()